In [9]:
!brew install libxml2 libxmlsec1 pkg-config

To reinstall 2.15.1_1, run:
  brew reinstall libxml2
To reinstall 1.3.9, run:
  brew reinstall libxmlsec1
To reinstall 2.5.1, run:
  brew reinstall pkgconf


In [6]:
!pip install xmlsec lxml

  Using cached xmlsec-1.3.17-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached xmlsec-1.3.17-cp313-cp313-macosx_11_0_arm64.whl (3.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.2 MB/s  0:00:00 22.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xmlsec]


In [2]:
import xmlsec
from lxml import etree

# 1. Create a sample XML document
xml_text = b"<Document><Secret>Because the knowledge of the secrets of the kingdom of heaven has been given to you, but not to them.</Secret></Document>"
root = etree.fromstring(xml_text)

# 2. Setup Encryption Context
# We use AES-256 for data and RSA-OAEP for the key
enc_ctx = xmlsec.EncryptionContext()

# Generate a random 256-bit (32 byte) AES session key
session_key = xmlsec.Key.generate(
    xmlsec.KeyData.AES, 256, xmlsec.KeyDataType.SESSION
)
enc_ctx.key = session_key

# 3. Encrypt the <Secret> element
target_node = root.find("Secret")

# This transforms the <Secret> tag into an <EncryptedData> block
encrypted_node = enc_ctx.encrypt_xml(target_node)

# 4. Wrap the session key (Hybrid Encryption)
# We load your public certificate to protect the session key
with open("certificate.pem", "rb") as f:
    public_key = xmlsec.Key.from_memory(f.read(), xmlsec.KeyFormat.CERT_PEM)

# Create an EncryptedKey node and add it to the document
enc_key_node = xmlsec.template.encrypted_data_create(
    root, xmlsec.Transform.AES256, type=xmlsec.EncryptionType.ELEMENT
)
xmlsec.template.encrypted_data_ensure_key_info(enc_key_node)
xmlsec.template.add_encrypted_key(enc_key_node, xmlsec.Transform.RSA_OAEP)

print("--- ENCRYPTED XML ---")
print(etree.tostring(root, pretty_print=True).decode())

TypeError: encrypt_xml() missing required argument 'node' (pos 2)

In [ ]:
# Setup Decryption Context
dec_ctx = xmlsec.EncryptionContext()

# Load the Private Key
with open("private_key.pem", "rb") as f:
    priv_key = xmlsec.Key.from_memory(f.read(), xmlsec.KeyFormat.PEM)
dec_ctx.key = priv_key

# Decrypt
# The library automatically finds the EncryptedKey, unlocks the session key,
# and then restores the original XML element.
decrypted_root = dec_ctx.decrypt(root)

print("--- DECRYPTED XML ---")
print(etree.tostring(root, pretty_print=True).decode())

In [7]:
import xmlsec
from lxml import etree

# 1. Setup Data and Template
xml_text = b"<Hospital><Patient>Jemmee</Patient><Diagnosis>Healthy</Diagnosis></Hospital>"
root = etree.fromstring(xml_text)
target = root.find("Diagnosis")

# Create template: AES-256 for data, RSA-OAEP for the key
enc_data = xmlsec.template.encrypted_data_create(
    root, xmlsec.Transform.AES256, type=xmlsec.EncryptionType.ELEMENT
)
ki = xmlsec.template.encrypted_data_ensure_key_info(enc_data)
ek = xmlsec.template.add_encrypted_key(ki, xmlsec.Transform.RSA_OAEP)

# 2. Load Keys
with open("certificate.pem", "rb") as f:
    pub_key = xmlsec.Key.from_memory(f.read(), xmlsec.KeyFormat.CERT_PEM)
with open("private_key.pem", "rb") as f:
    priv_key = xmlsec.Key.from_memory(f.read(), xmlsec.KeyFormat.PEM)

# 3. ENCRYPTION
enc_ctx = xmlsec.EncryptionContext()

# Step A: Generate a session key (the "Secret" that encrypts the data)
session_key = xmlsec.Key.generate(xmlsec.KeyData.AES, 256, xmlsec.KeyDataType.SESSION)

# Step B: Use the Public Key to encrypt that session key into the <EncryptedKey> node
enc_ctx.key = pub_key
enc_ctx.encrypt_key(ek, session_key)

# Step C: Use the session key to encrypt the actual XML data
enc_ctx.key = session_key
enc_ctx.encrypt_xml(enc_data, target)

# Replace original with encrypted
parent = target.getparent()
parent.replace(target, enc_data)

print("--- ENCRYPTED XML ---")
print(etree.tostring(root, pretty_print=True).decode())

# --- DECRYPTION ---
dec_ctx = xmlsec.EncryptionContext()
# The Decrypter needs the private key to unwrap the session key
dec_ctx.key = priv_key

# This one call handles the full "unwrap and decrypt" process
dec_ctx.decrypt(enc_data)

print("\n--- DECRYPTED XML ---")
print(etree.tostring(root, pretty_print=True).decode())

AttributeError: 'xmlsec.EncryptionContext' object has no attribute 'encrypt_key'